# 南島語料搜尋小工具

這學期修了「語言田野調查」，請了汶水泰雅語的族語老師來給我們上課，並將每個禮拜採集的語料整理成一個.docx檔。

## 語料檔範例

```
1.
ma-tas=sami         su     tigami     ni     Payan    babay   i    Laway
AF-write=1PL.NOM    ACC    letter     BEN    Payan    for     NOM  Laway
主焦-寫=1PL.主格     受格    信         受益格   Payan    給      主格  Laway

#e I write the letter to Laway with Payan.  
#c 我跟Payan一起寫信給Laway。   
#n 比較 ki Payan 和 ni Payan: ki Payan的意思為「和 Payan」，ni Payan的意思為「 Payan的信」。

2.
si-pa-quwas=mu           i      yaya
CF-VBL-song=1SG.GEN      NOM    mother
參焦-動化-歌=1SG.屬格      名詞    媽媽
 
#e I sing for mom.
#c 我唱歌給媽媽聽。
#n i 可以省略。
```

## 語料編碼格式

```
[編號].
[族語轉寫]
[英文Glossing]
[中文Glossing]
[空行]
#e [英文翻譯]
#c [中文翻譯]
#n [註釋]
[空行]
```

但現在我遇到的麻煩是，我好奇的某個語言現象常常散佈在好多個`.docx`檔中，變成我要一個一個打開，然後用搜尋功能一個一個找出來。所以我就在想，是否可以寫個小程式，讓`python`爬完我資料夾中的`.docx`檔後，整理成單一的資料結構，這樣就可以直接下個搜尋，撈出所有相關的語料。

既然要處理word檔，我找到了一個叫做`docx`的套件，可以進去每個word檔撈出裡頭的文字。

## 程式碼

In [3]:
import sys
import re
import os
from docx import Document
import pandas as pd
from IPython.display import display, HTML

class CorpusProcessor:
    def __init__(self, path):
        self.result = []
        self.path = path
        self._process_path(self.path)
        
    def search(self, query_word):
        """
        query_word: <Str>
            要搜索的字串，此搜索字串會先丟入re.compile()
            因此可以接收regex string
            例如此參數可以輸入"(LOC|INS)"
            則可以給出所有標有LOC或INS的語料。
        """

        pattern = re.compile(query_word)

        r = self._query_keyword(pattern)
        
        for k, v in r.items():
            print(f"<< {k} >>")
            if len(v) == 0:
                print("本週無相關資料")
            else:
                for item in v:
                    n = item['num']
                    c = item['content']
                    cc = c.split('\n')
                    cc = list(filter(lambda x: x!='', cc))
                    display(pd.DataFrame([cc[0].split(), cc[1].split(), cc[2].split()]).rename({0: "泰雅：", 1: "英文：", 2: "中文："}, axis='index')) 
                    print("[英文翻譯]")
                    print([ccc for ccc in cc if ccc.startswith("#e")][0])
                    print("[中文翻譯]")
                    print([ccc for ccc in cc if ccc.startswith("#c")][0])
                    print("[註釋]")
                    print([ccc for ccc in cc if ccc.startswith("#n")][0])
    
    def _process_path(self, p):
        for filename in os.listdir(p):
            if not filename.startswith("~") and (filename.endswith(".docx") or filename.endswith(".doc")):
                document = self._open_file(filename)
                self.result.append(self._process(document, filename))

    def _open_file(self, name):
        d = Document(name)
        return d
    
    
    def _process(self, doc, filename):
        result = {
            "file_name": filename,
            "data": []
        }
        all_p = doc.paragraphs
        num_re = re.compile("(\d{1,2})\.")

        num = 0
        current_index = -1
        # rrr = map(lambda x: re.match(num_re, x.text), all_p)
        start = False
        for p in all_p:
            num_re = re.compile("(\d{1,2})\.")
            match = re.match(num_re, p.text)

            if match:
                start = True
                num = match.group(1)
                result["data"].append({"num": num, "content": ""})
                current_index = len(result["data"]) - 1
            else:
                if not start:
                    pass
                else:
                    result["data"][current_index]["content"] += "\n" + p.text

        return result
    
    def _query_keyword(self, q):
        result = {}

        for file in self.result:
            result[file["file_name"]] = [sentence for sentence in file["data"] if re.search(q, sentence["content"])]

        return result

## 展示時間

要使用這個程式就必須要有以上述方式編碼的`.docx`語料檔。

這學期我的期末報告打算做Causative的句法議題，因此就來看一下目前收集到的汶水泰雅語中所有的使動結構：

In [4]:
corpus = CorpusProcessor('.')
corpus.search('CAUS')

<< W10_20190507_HW.docx >>


,0,1,2,3,4
泰雅：,si-pa-kasiyuk=mu,ku,pa-patas,i,rawin=mu
英文：,CF-CAUS-borrow=1SG.GEN,NOM,NML-write,NOM,friend=1SG.GEN
中文：,參焦-使動-借=1SG.屬格,主格,名化-寫,主格,朋友=1SG.屬格


[英文翻譯]
#e I lend pen to my classmate.
[中文翻譯]
#c 我借筆給我同學。
[註釋]
#n ku pa-patas 和 i rawin=mu可互換。這句「讓」的意思比較明顯。「我肯讓你向我借東西」，有點免搶。借的物品為主詞。


,0,1,2,3,4
泰雅：,pa-kasiyuk-an=mu,ku,rawin=mu,su,pa-patas
英文：,CAUS-borrow-LF=1SG.GEN,NOM,friend=1SG.GEN,ACC,pa-write
中文：,使動-借-處焦=1SG.屬格,主格,朋友=1SG.屬格,受格,pa-寫


[英文翻譯]
#e I lend pen to my classmate.
[中文翻譯]
#c我借筆給我同學。
[註釋]
#n ditransitive 三論元句型。ku  rawin=mu 和su  pa-patas可互換。「我借給你」借的人為主詞。


...

[英文翻譯]
#e I lent him him this pen last month.
[中文翻譯]
#c 這隻筆是我上個月借給他的。
[註釋]
#n maquwa ka watin : 下個月，要來的月。


,0,1,2,3,4
泰雅：,si-pi-kitaal=kung,su,singbung,ni,yaya
英文：,CF-CAUS-taal=1SG.NOM,ACC,news,GEN,mother
中文：,參焦-使動-看=1Sg.主格,受格,新聞,屬格,媽媽


[英文翻譯]
#e Mother lets me see news.
[中文翻譯]
#c 媽媽讓我看新聞。
[註釋]
#n si-pa-kitaal = si-pi-kitaal=si-ka-kitaal。 su singbung 和 ni yaya 可以互換位址。


,0,1,2,3,4,5,6,7
泰雅：,ini,pa-kital-i,su,singbung,ni,yaya,i,yaba
英文：,NEG,CAUS-see-CF,ACC,news,GEN,mother,NOM,father
中文：,否定,使動-看-參焦,受格,新聞,屬格,媽媽,主格,爸爸


[英文翻譯]
#e Mother doen’t let father watch news.
[中文翻譯]
#c 媽媽不讓爸爸看新聞。
[註釋]
#n 這裡paki-tal-i 可說成 piki-tal-i
<< W5_20190320_HW.docx >>
本週無相關資料
<< W7_20190327_HW.docx >>
本週無相關資料


讓我們看一下泰雅語的「煮」：

In [7]:
corpus.search('cook')

<< W10_20190507_HW.docx >>
本週無相關資料
<< W8_20190417_HW.docx >>
本週無相關資料
<< W4_20190313_HW.docx >>


,0,1,2,3
泰雅：,uniyan=su,t<um>ahuk,su,ayang
英文：,PROG=2SG.GEN,<AF>cook,ACC,soup
中文：,正在=2SG.屬格,煮,受格,湯


[英文翻譯]
#e You are making soup. 
[中文翻譯]
#c 你正在煮湯。
[註釋]
#n tumahuk煮什麼湯都可以，有水的就可以。
<< W9_20190424_HW.docx >>
本週無相關資料
<< W6_20190327_HW.docx >>
本週無相關資料
<< W11_20190508_HW.docx >>


,0,1,2,3
泰雅：,si-pa-hapuy=nia,nanak,i,hiya
英文：,CF-VBL-fire=3SG.GEN,alone,NOM,3SG.NOM
中文：,參焦-動化-火=3SG.屬格,獨自,主格,3SG.主格


[英文翻譯]
#e he cooks for himself.
[中文翻譯]
#c 他煮飯給自己吃。
[註釋]
#n i nia和nanak位置可以交換。
<< W5_20190320_HW.docx >>
本週無相關資料
<< W7_20190327_HW.docx >>


...

[英文翻譯]
#e I am boiling soup in the kitchen.
[中文翻譯]
#c 我在廚房裡煮湯。
[註釋]
#n 


,0,1,2,3,4
泰雅：,t<um>ahuk=su,ayang,i,hahapuy-an,qu
英文：,<AF>cook=2SG.NOM,soup,LOC,cook-LOC,Q
中文：,<主焦>煮=2SG.主格,湯,處格,煮-處格,疑問助詞


[英文翻譯]
#e Do you make soup in the kitchen?
[中文翻譯]
#c 你在廚房裡煮湯嗎？
[註釋]
#n 請與下句在句首加了uni’an做比較。


,0,1,2,3,4,5,6
泰雅：,uni’an=su,t<um>ahuk,su,ayang,i,hahapuyan,qu
英文：,PROG=2SG.NOM,<AF>cook,ACC,soup,LOC,cook-LOC,Q
中文：,進行=2SG.主格,<主焦>煮湯,受格,湯,處格,煮-處格,疑問助詞


[英文翻譯]
#e Are you making soup in the kitchen?
[中文翻譯]
#c 你在廚房裡煮湯嗎？
[註釋]
#n 本句中第一個su為第二人稱的clitic，第二個su為受格標記。


query string也可以接收regex string，假設我想叫出所有含有「工具格」(Instrumental case)和「共格」(Comitative case)的句子:

In [13]:
corpus.search('共格|工具格')

<< W10_20190507_HW.docx >>


,0,1,2,3,4
泰雅：,ma-tas,na,pa-patas,i,hiya
英文：,AF-write,INS,NML-write,NOM,3SG.NOM
中文：,主焦-寫,工具格,名化-寫,主格,3SG.主格


[英文翻譯]
#e He writes with pen.
[中文翻譯]
#c他用筆寫字。
[註釋]
#n 


,0,1,2
泰雅：,si-patas=mu,na,pa-patas
英文：,CF-write=1SG.GEN,INS,NML-write
中文：,參焦-寫=1SG.屬格,工具格,名化-寫


[英文翻譯]
#e I write with pen.
[中文翻譯]
#c 我用筆寫字。
[註釋]
#n AF的p/m型，原形為matas，但加了前綴會變成patas(e.g. si + matas -> sipatas)


...

[英文翻譯]
#e I’ve already sold pen to Payan.
[中文翻譯]
#c 我把筆賣給Payan了。
[註釋]
#n 


,0,1,2,3,4
泰雅：,pa-b<in>asun=mu,ku,pa-patas,ki,Payan
英文：,IRR-<PF>buy=1SG.GEN,NOM,pa-write,COM,Payan
中文：,非實現-<受焦>買=1SG.屬格,主格,pa-寫,共格,Payan


[英文翻譯]
#e I will sell pen to Payan. / I will buy pen from Payan.
[中文翻譯]
#c 我要把筆賣給Payan。/ 我要跟Payan買筆。
[註釋]
#n 
<< W5_20190320_HW.docx >>
本週無相關資料
<< W7_20190327_HW.docx >>


,0,1,2,3,4,5,6
泰雅：,uni’an,i,humibaw,su/*ku,paruxau,i,yaya
英文：,PROG,LNK,cut,ACC,vegetable,NOM,mother
中文：,進行,連繫詞,切,受格,菜,主格,媽媽


[英文翻譯]
#e Mother is cutting vegetable.
[中文翻譯]
#c媽媽正在切菜。
[註釋]
#n 若不用特別強調「正在」，可以省略句首的”uni’an i”。humibaw意義為「切」，而上句的hahibaw 的意義為「用來切」(工具格？)。


## 心得

這學期的語言田調課，感覺很像是上學期句法課學到的知識終於派上了用場，而且還是用在一個雖然從來不認識但其實離自己其實很近的活生生的南島語上。好像課本上那些抽象的術語都有了血肉之軀，但當然，實際採集到的語料（或者說實際的語言）絕對不會是像每本參考語法上寫的那樣那麼方正、那麼規矩，不是每個實際的句子都能無誤地由某幾條規則推導出那樣，而那些不規則性所反映的正是實際語言在演化/涵化/變化的痕跡以及人腦的複雜性。

後來覺得這樣的小程式好像就是一個小語料庫那樣，從採收來的一籃藍語料裡挑出想要的果實。至此我也才體會到上學期的語料庫語言學課上，老師不斷強調的，語料庫真正重要的是經過分析整理後的標記，如果沒有了這些用沒日沒夜的語料轉寫辛勞換來的標記，我根本不可能透過這個程式去抓出所有的屬格或者工具格，然後做進一步的語言學分析，更不用說不同人對同個語料可能會有不同的標記和詮釋。

突然想起這學期遇到的另一位老師，他總是對當今的機器/AI的能力持疑。現在的確是有很多自動標記詞性的演算法，無論是用統計方法或是機率模型，但對於像泰雅語這種電子化的語料非常少數的語言，常常連我在判斷某個詞到底要標什麼格位時都無法60%確定了，我又要如何寫演算法讓機器幫忙找出呢？或者是如果沒有語言學家先給定泰雅語裡面到底名詞有幾種格位，電腦真的能夠自己找出有幾種嗎？或者這真的是個有標準答案的問題嗎？常常聽到那位老師這類悲觀的論調（以及對人類族群抱以樂觀的論調）時，就會想說，那麼我們要做的應該不是從此封印電腦吧，而是要想辦法讓電腦在這些層面能「跟得上」人類。而要怎麼讓電腦「像人類一樣」，似乎又必須回到諸多計算模型的模擬，我們不一定要讓電腦給出最佳唯一解，但至少應該要讓電腦能像我們一樣能夠存疑和問問題才對。

這大概就是本週的微心得。

我有把這個微專案放到[Github](https://github.com/puerdon/corpus_processor)上，目前是由我和力行一起維護，目前也還在想可以加什麼功能，能讓期末報告寫得更順利。